# Test THREDDS using Magpie Authentication/Authorization against Twitcher proxy.


### **NOTE:**

Because cells gradually apply new permissions to validate that modified permissions on resources get applied and processed as expected, previous cells re-executed non-sequentially could result into different resource access decisions. This will make previously successful tests fail on following runs. The test should be re-executed from the start with new test user accounts or following test data cleanup to re-evaluate the results acccordingly.


## Configuration parameters

In [1]:
# NBVAL_IGNORE_OUTPUT

from __future__ import print_function

import json

import os
import uuid
import requests
print("Setup configuration parameters...")

PAVICS_HOST = (os.getenv("PAVICS_HOST") or "pavics.ouranos.ca").rstrip("/")
if not PAVICS_HOST:
    raise ValueError("Cannot run test without a PAVICS_HOST value.")

PAVICS_URL = "https://{}".format(PAVICS_HOST)
VERIFY_SSL = True if "DISABLE_VERIFY_SSL" not in os.environ else False
MAGPIE_URL = os.getenv("MAGPIE_URL") or (PAVICS_URL + "/magpie")
TWITCHER_PROXY = "/twitcher/ows/proxy"
TWITCHER_URL = os.getenv("TWITCHER_URL") or (PAVICS_URL + TWITCHER_PROXY)
THREDDS_SERVICE = "thredds"
THREDDS_URL = "{}/{}".format(TWITCHER_URL, THREDDS_SERVICE)

HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

MAGPIE_ANONYMOUS_GROUP = "anonymous"
TEST_MAGPIE_DISABLE_CLEANUP = str(os.getenv("TEST_MAGPIE_DISABLE_CLEANUP", False)).lower() in ["true", "1", "on"]
TEST_MAGPIE_ADMIN_USERNAME = os.getenv("TEST_MAGPIE_ADMIN_USERNAME")
TEST_MAGPIE_ADMIN_PASSWORD = os.getenv("TEST_MAGPIE_ADMIN_PASSWORD")
if not TEST_MAGPIE_ADMIN_USERNAME or not TEST_MAGPIE_ADMIN_PASSWORD:
    raise ValueError("Missing test admin credentials to run tests.")

TEST_GROUP_NAME = os.getenv("TEST_MAGPIE_THREDDS_GROUP") or "test-auth-{!s}".format(uuid.uuid4())
TEST_USER_NAME = os.getenv("TEST_MAGPIE_THREDDS_USERNAME") or "test-user-{!s}".format(uuid.uuid4())
# password must not be displayed to keep minimal obfuscation
#   since test user name will be logged, and eventually receive slightly more access than 'anonymous',
#   don't give a chance for anyone to guess the credentials
TEST_USER_PWD = os.getenv("TEST_MAGPIE_THREDDS_PASSWORD") or str(uuid.uuid4())

print("  Will use Magpie URL:   [{}]".format(MAGPIE_URL))
print("  Will use Twitcher URL: [{}]".format(TWITCHER_URL))
print("  Will use THREDDS URL:  [{}] (service: {})".format(THREDDS_URL, THREDDS_SERVICE))


def response_msg(message, response, is_json=True):
    """Append useful response details to provided message."""
    read_json = False
    _body = response.text
    _detail = "<unknown>"
    if is_json:
        # magpie formatted error
        try:
            _body = response.json()
            _detail = _body.get("detail", _body.get("message", "<unknown>"))
            read_json = True
        except json.JSONDecodeError:
            pass  # ignore and revert to text body since it could not be parsed as JSON
    if not read_json:
        # twitcher formatted error
        if "<ExceptionText>" in _body:
            _detail = _body.split("<ExceptionText>")[-1].split("</ExceptionText>")[0].strip()
        # thredds formatted error
        elif "Error {" in _body:
            _detail = _body.split("message = ")[-1].split(";\n")[0].strip()
    return "{} Response replied with ({}) [{}]\nContent:\n\n".format(message, response.status_code, _detail, _body)

# test URLs are reachable
resp = requests.get(MAGPIE_URL, verify=VERIFY_SSL, headers=HEADERS)
if not resp.status_code == 200:
    raise ValueError(response_msg("Could not reach MAGPIE_URL.", resp))
resp = requests.get(THREDDS_URL, verify=VERIFY_SSL)
if not resp.status_code == 200:
    raise ValueError(response_msg("Could not reach THREDDS_URL.", resp))


Setup configuration parameters...
  Will use Magpie URL:   [http://localhost:2001]
  Will use Twitcher URL: [http://localhost:8001/ows/proxy]
  Will use THREDDS URL:  [http://localhost:8001/ows/proxy/thredds] (service: thredds)


## Validate test admin user can have needed access to adjust permissions and that anonymous group is valid

In [2]:
# NBVAL_IGNORE_OUTPUT

print("Validate that user has needed admin-level access... ", end="")

def login_user(username, password):
    _data = {"user_name": username, "password": password}
    _path = MAGPIE_URL + "/signin"
    _resp = requests.post(_path, json=_data, verify=VERIFY_SSL, headers=HEADERS)
    if _resp.status_code != 200:
        raise ValueError(response_msg("\nCould not login with provided test admin credentials.", _resp))
    return _resp.cookies

ADMIN_COOKIES = login_user(TEST_MAGPIE_ADMIN_USERNAME, TEST_MAGPIE_ADMIN_PASSWORD)

# test admin-level access with route that is admin only
path = MAGPIE_URL + "/users"
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    raise ValueError(response_msg("\nProvided user credentials doesn't have administrative permissions.", resp))

path = "{}/groups/{}".format(MAGPIE_URL, MAGPIE_ANONYMOUS_GROUP)
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    msg = "\nCould not validate Magpie anonymous group using name: [{}]".format(MAGPIE_ANONYMOUS_GROUP)
    raise ValueError(response_msg(msg, resp))

print("OK")

Validate that user has needed admin-level access... OK


## Define functions to clean test user and group after tests

Since we know that admin is correctly configured up to this point,
setup cleanup operations to remove test data after test execution
(unless `TEST_MAGPIE_DISABLE_CLEANUP = true` was specified).

In [3]:
# NBVAL_IGNORE_OUTPUT

CLEANUP_CALLED = False

def cleanup_test_data(skip_fail=True):
    """Cleanup test elements created during the test.

    Only raise at the complete end to attempt removal of as much element as possible in case of failures.
    """
    global TEST_MAGPIE_DISABLE_CLEANUP
    global CLEANUP_CALLED
    if TEST_MAGPIE_DISABLE_CLEANUP:
        print("WARNING - Skipping test data cleanup because of TEST_MAGPIE_DISABLE_CLEANUP option!")
        return

    CLEANUP_CALLED = True
    failed = False
    for item_type, item_name, item_value in [
        ("users", "TEST_USER_NAME", TEST_USER_NAME),
        ("groups", "TEST_GROUP_NAME", TEST_GROUP_NAME),
    ]:
        try:
            _path = "{}/{}/{}".format(MAGPIE_URL, item_type, item_value)
            _resp = requests.delete(_path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
            _head = "Cleanup of {} [{}]:".format(item_name, item_value)
            if _resp.status_code == 200:
                print(_head, "OK")
            elif _resp.status_code == 404:
                print(_head, "WARNING - does not exist (maybe it failed to be created in the first place?)")
                failed = True  # most definitely the test is already failing at this point anyway
            else:
                print(_head, "ERROR - could not be removed")
                failed = True
        except Exception as exc:
            print("Cleanup raised: [{}]".format(exc))
            failed = True
    if not skip_fail and failed:
        raise ValueError("Could not cleanup all test data.")


## Create a test group and test user that will not have admin access

In [4]:
# NBVAL_IGNORE_OUTPUT

print("Create test group and test user... ")
print("  Will use TEST_USER_NAME:  [{}]".format(TEST_USER_NAME))
print("  Will use TEST_GROUP_NAME: [{}]".format(TEST_GROUP_NAME))

data = {"group_name": TEST_GROUP_NAME}
path = MAGPIE_URL + "/groups"
resp = requests.post(path, json=data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not create test group [{}]".format(TEST_GROUP_NAME), resp))

data = {"user_name": TEST_USER_NAME, "password": TEST_USER_PWD,
        "email": "{}@mail.com".format(TEST_USER_NAME), "group_name": TEST_GROUP_NAME}
path = MAGPIE_URL + "/users"
resp = requests.post(path, json=data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not create test user [{}]".format(TEST_USER_NAME), resp))

print("Creation: OK")

Create test group and test user... 
  Will use TEST_USER_NAME:  [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Will use TEST_GROUP_NAME: [test-auth-113a7553-cc63-4309-9419-5d8f1ac3fb2e]
Creation: OK


## Create specific test resources under THREDDS service

Expected data structure as per:
    [https://github.com/bird-house/birdhouse-deploy/blob/master/birdhouse/scripts/bootstrap-testdata](
    https://github.com/bird-house/birdhouse-deploy/blob/master/birdhouse/scripts/bootstrap-testdata
    )

In [5]:
# NBVAL_IGNORE_OUTPUT

print("Create test resources under THREDDS service... ", end="")

path = MAGPIE_URL + "/services/{}".format(THREDDS_SERVICE)
resp = requests.get(path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
if resp.status_code != 200:
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not fetch THREDDS service details [{}].".format(THREDDS_SERVICE), resp))
thredds_res_id = resp.json()["service"]["resource_id"]

def create_or_get_resource(resource_name, resource_type, parent_id):
    _path = MAGPIE_URL + "/resources"
    _data = {"parent_id": parent_id, "resource_type": resource_type, "resource_name": resource_name}
    _resp = requests.post(_path, json=_data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
    _msg = "[name: {}, type: {}, parent: {}].".format(resource_name, resource_type, parent_id)
    if _resp.status_code == 409:
        _path = "{}/{}".format(_path, parent_id)
        _resp = requests.get(_path, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
        if _resp.status_code != 200:
            cleanup_test_data()
            raise ValueError(response_msg("Could not retrieve resource expected to exist {}.".format(_msg), _resp))
        _resource = _resp.json()["resource"]
        _children = _resource["children"]
        for _, _child_info in _children.items():
            if _child_info["resource_name"] == resource_name:
                return _child_info
    elif _resp.status_code != 201:
        cleanup_test_data()
        raise ValueError(response_msg("\nCould not create resource {}.".format(_msg), _resp))
    else:
        return _resp.json()["resource"]
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not create or retrieve resource {}.".format(_msg), _resp))

birdhouse_res = create_or_get_resource("birdhouse", "directory", thredds_res_id)
testdata_res = create_or_get_resource("testdata", "directory", birdhouse_res["resource_id"])
secure_res = create_or_get_resource("secure", "directory", testdata_res["resource_id"])
taskmax_name = "tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc"
taskmax_res = create_or_get_resource(taskmax_name, "file", secure_res["resource_id"])

birdhouse_path = birdhouse_res["resource_name"]
birdhouse_catalog = "catalog/{}/catalog.html".format(birdhouse_path)
testdata_path = "{}/{}".format(birdhouse_path, testdata_res["resource_name"])
testdata_catalog = "catalog/{}/catalog.html".format(testdata_path)
secure_path = "{}/{}".format(testdata_path, secure_res["resource_name"])
secure_catalog = "catalog/{}/catalog.html".format(secure_path)
taskmax_path = "{}/{}".format(secure_path, taskmax_res["resource_name"])
taskmax_file = "fileServer/{}".format(taskmax_path)
taskmax_ncml = "ncml/{}".format(taskmax_path)
taskmax_dodsC = "dodsC/{}.html".format(taskmax_path)

print("OK\n")
for res in [birdhouse_path, birdhouse_catalog,
            testdata_path, testdata_catalog,
            secure_path, secure_catalog,
            taskmax_path, taskmax_file, taskmax_ncml, taskmax_dodsC]:
    print("Resource: [{}]".format(res))

Create test resources under THREDDS service... OK

Resource: [birdhouse]
Resource: [catalog/birdhouse/catalog.html]
Resource: [birdhouse/testdata]
Resource: [catalog/birdhouse/testdata/catalog.html]
Resource: [birdhouse/testdata/secure]
Resource: [catalog/birdhouse/testdata/secure/catalog.html]
Resource: [birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
Resource: [fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
Resource: [ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
Resource: [dodsC/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc.html]


## Create specific permissions for THREDDS resources to block public access to 'secure' directory

In [6]:
# NBVAL_IGNORE_OUTPUT

print("Create permissions on test resources... ", end="")

def set_permission(permission, resource, target_type, target_name):
    _res_id = resource["resource_id"]
    _path = "{}/{}/{}/resources/{}/permissions".format(MAGPIE_URL, target_type, target_name, _res_id)
    _data = {"permission": permission}
    _resp = requests.put(_path, json=_data, verify=VERIFY_SSL, cookies=ADMIN_COOKIES, headers=HEADERS)
    if _resp.status_code not in [200, 201]:
        _msg = "\nCleanup called before? {} (if called, following error could be expected)".format(CLEANUP_CALLED) + \
               "\nCould not set permission [{}] for [{}, {}] over resource [{}, {}]" \
               .format(permission, target_type, target_name, resource["resource_name"], _res_id)
        cleanup_test_data()
        raise ValueError(response_msg(_msg, _resp))


# these permissions should be already defined by following components, but guarantee they are created here:
#   https://github.com/bird-house/birdhouse-deploy/tree/master/birdhouse/optional-components/all-public-access
#   https://github.com/bird-house/birdhouse-deploy/tree/master/birdhouse/optional-components/secure-thredds
set_permission("browse-allow-recursive", birdhouse_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("read-allow-recursive", birdhouse_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("browse-deny-recursive", secure_res, "groups", MAGPIE_ANONYMOUS_GROUP)
set_permission("read-deny-recursive", secure_res, "groups", MAGPIE_ANONYMOUS_GROUP)

print("OK")

Create permissions on test resources... OK


## Verify allowed/blocked user access to secured resources

The initial configuration will by default grant full access to administrators.

Only public resources will be accessible by anyone. Therefore, anything under `secure` directory will be blocked to both the anonymous identify (no user logged in), and the specific test user. Being logged in as the test user does not provide any more access at this point than simply not being logged in at all.

#### **Note 1**

Resources allowed/denied access result could differ if `thredds` is not correctly configured since custom configuration is effective.
If errors occur, make sure that `configuration` settings from `GET {MAGPIE_URL}/services/thredds` response correspond to `configuration` definition under the corresponding service applied onto the server from: https://github.com/bird-house/birdhouse-deploy/blob/master/birdhouse/config/magpie/providers.cfg.template

#### **Note 2**

Because permissions are applied onto Magpie and are resolved for access by Twitcher instead, any active caching of older requests is not yet synchronized on Twitcher side right after the permission update on Magpie side (caches are not shared, and therefore not invalidated on permission update). Following requests to the same resources (within the caching expiration delay) will hit the cached response triggered by the access during the previous requests. New permissions resolution will not be effective until the cache expires. For this reason, we explicitly ask (via request headers) to ignore caches during those resource access requests to bypass the synchronization problem. Caching is not a critical component performance-wise for one-shot requests, and can be safely disabled for this test.


In [7]:
# NBVAL_IGNORE_OUTPUT

print("Testing allowed/blocked access to secured THREDDS resources...")

TEST_USER_COOKIES = login_user(TEST_USER_NAME, TEST_USER_PWD)
ANONYMOUS_COOKIES = {}

# avoid potential leak of critical admin/anonymous credentials for all following prints by using fill-in values
TEST_ADMIN_NAME = "<TEST_MAGPIE_ADMIN_USERNAME>"
TEST_ANONYM_NAME = "<ANONYMOUS_USER>"


def has_access(resource_path, user_cookies, user_name):
    _header_ignore_cache = {"Cache-Control": "no-cache"}
    _path = "{}/{}".format(THREDDS_URL, resource_path)
    _resp = requests.get(_path, verify=VERIFY_SSL, cookies=user_cookies, headers=_header_ignore_cache)
    _code = _resp.status_code
    if _code in [200, 401, 403]:
        is_allowed = _code == 200
        str_allowed = "Allowed" if is_allowed else "Denied"
        # for debugging and cleanup, print its generated name, but not the password
        print("Detail:\n"
              "  Resource: [{}]\n"
              "  User:     [{}]\n"
              "  Code:     [{}]\n"
              "  Access:   [{}]".format(_path, user_name, _code, str_allowed))
        return is_allowed
    cleanup_test_data()
    print("Detail:\n"
          "  Resource: [{}]\n"
          "  User:     [{}]\n"
          "  Code:     [{}]\n"
          "  Access:   [Error]".format(_path, user_name, _code))
    _msg = "Unexpected status code during access to resource [{}]".format(_path)
    raise ValueError(response_msg(_msg, _resp, is_json=False))

# admin always has full access regardless of permissions
assert has_access(birdhouse_catalog, ADMIN_COOKIES, TEST_ADMIN_NAME)
assert has_access(testdata_catalog, ADMIN_COOKIES, TEST_ADMIN_NAME)
assert has_access(secure_catalog, ADMIN_COOKIES, TEST_ADMIN_NAME)
assert has_access(taskmax_file, ADMIN_COOKIES, TEST_ADMIN_NAME)
assert has_access(taskmax_ncml, ADMIN_COOKIES, TEST_ADMIN_NAME)
assert has_access(taskmax_dodsC, ADMIN_COOKIES, TEST_ADMIN_NAME)

# test user has different access depending on permissions applied
assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)
assert not has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)

assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)

print("Expected secured access to all resources: OK")

Testing allowed/blocked access to secured THREDDS resources...
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [<TEST_MAGPIE_ADMIN_USERNAME>]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  U

## Verify that directories become browsable after update of test user group access

Users that are members of the allowed test group will be allowed to `browse` contents (even if not administrators).

Read (fetch of files marked as data) remains blocked for now because only `browse` is granted to this test group.

Anonymous user will still not have access to resources under `secure` as its is not a member of the test group.

#### **Note**

What defines an endpoint to require `browse` vs `read` access is according to THREDDS service configuration in Magpie (see previous cell Note).


In [8]:
# NBVAL_IGNORE_OUTPUT

set_permission("browse-allow-recursive", secure_res, "groups", TEST_GROUP_NAME)

assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)     # becomes accessible
assert has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)       # becomes accessible (ncml = browse)
assert not has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)   # still blocked (fileServer = read)
assert not has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)  # still blocked (dodsC = read)

# following remains the same as previous cell, but validate to make sure
assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)  # still blocked anything under 'secure'
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)

print("Expected browse access to all resources: OK")


Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-20061

## Verify that secured directory contents become readable after update of test user group access

By adding `read` access specifically to test group recursively onto the `secure` directory, all resources under that directory should now become accessible by the test user (via its group memberships), as both `browse` set in previous cell and newly added `read` permissions are obtained.

In [9]:
# NBVAL_IGNORE_OUTPUT

# Additionally grant read access, test-user should now be allowed to fetch of files marked as data
set_permission("read-allow-recursive", secure_res, "groups", TEST_GROUP_NAME)

# Combined permissions resolution grant full access to users member of test group
assert has_access(birdhouse_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(testdata_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(secure_catalog, TEST_USER_COOKIES, TEST_USER_NAME)
assert has_access(taskmax_ncml, TEST_USER_COOKIES, TEST_USER_NAME)    # up to here, remains the same (previously set browse)
assert has_access(taskmax_file, TEST_USER_COOKIES, TEST_USER_NAME)    # becomes accessible (fileServer = read)
assert has_access(taskmax_dodsC, TEST_USER_COOKIES, TEST_USER_NAME)   # becomes accessible (dodsC = read)

# following remains the same as previous cell, but validate to make sure
assert has_access(birdhouse_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert has_access(testdata_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(secure_catalog, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)  # still blocked anything under 'secure'
assert not has_access(taskmax_ncml, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(taskmax_file, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)
assert not has_access(taskmax_dodsC, ANONYMOUS_COOKIES, TEST_ANONYM_NAME)

print("Expected read access to all resources: OK")

Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/catalog.html]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/testdata/catalog.html]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/catalog/birdhouse/testdata/secure/catalog.html]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/ncml/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-200612.nc]
  User:     [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]
  Code:     [200]
  Access:   [Allowed]
Detail:
  Resource: [http://localhost:8001/ows/proxy/thredds/fileServer/birdhouse/testdata/secure/tasmax_Amon_MPI-ESM-MR_rcp45_r2i1p1_200601-20061

In [10]:
# NBVAL_IGNORE_OUTPUT

# final cleanup, everything is expected to have worked up to here,
# so force failure to ensure we return to original 'clean' state
cleanup_test_data(skip_fail=False)
print("All tests: OK")


Cleanup of TEST_USER_NAME [test-user-58ec4722-afcd-4f90-95a9-3ccee8dc162b]: OK
Cleanup of TEST_GROUP_NAME [test-auth-113a7553-cc63-4309-9419-5d8f1ac3fb2e]: OK
All tests: OK
